written by chatGPT. The code quantify the contacts made by each probe with a given chromosomal region. It uses the output of the ssHiC pipeline: a table with 3 columns specifying chr, chr_coord, and genome_coord, and the rest of the column being the contact made by each targeted fragments

In [129]:
import pandas as pd
import numpy as np

In [166]:
input_dir_contacts = "/mnt/e/Science/ssHiC/Samples/"
output_dir = "/mnt/d/Science/Owncloud/Experiments/Capture/Analysis/Intra_Inter_Cis_Coverage_Chr/cis_side_specific/"

suffix = "_sub4M_S288c_DSB_LY_Capture_artificial_v8_cutsite_q20_PCRfree"
reference_table = pd.read_csv(str(output_dir) + "_ssHiC_probe_location_and_side.tsv", sep="\t")

samples = ["AD206", "AD207", "AD208", "AD209", "AD212", "AD213"] #to do
samples = ["AD162",  "AD210", "AD211",  "AD233", "AD234", "AD235", "AD236", 
           "AD237", "AD238", "AD239", "AD240", "AD241", "AD242", "AD243", "AD244", "AD245", "AD246", 
           "AD247", "AD248", "AD257", "AD258", "AD259", "AD260", "AD289", "AD290", "AD291", "AD292", 
           "AD293", "AD294", "AD295", "AD296", "AD297", "AD298", "AD299", "AD300", 
           "AD301", "AD302", "AD342", "AD343", "AD344", "AD345", "AD346", "AD347", "AD401", "AD402", 
           "AD403", "AD404", "AD405", "AD406", "AD407", "AD412", "AD413", "AD414", "AD432", "AD433m",
           "AD446", "AD447", "AD448", "AD449", "AD450", "AD451", "AD452", "AD453", "AD454", "AD455",
           "AD456", "AD457", "AD458", "AD459", "AD460", "AD461", "AD462", "AD463", "AD464", "AD465",
           "AD473", "AD474", "AD475", "AD476", "AD477", "AD478", "AD479", "AD480", "AD481", "AD488",
           "AD489", "AD490", "AD491", "AD492", "AD493", "AD542", "AD543", "AD544", "AD545", "AD546",
           "AD547", "AD548", "AD549", "AD550", "AD551", "AD552", "AD553", "AD554", "AD555", "AD570",
           "AD571", "AD572", "AD573", "AD574", "AD575", "AD576", "AD577", "AD586", "AD587", "AD588",
           "AD589", "AD590", "AD591", "AD592", "AD593"]


interval = 25000 #in bp

This function calculates the sum of contacts immediately in cis and on the cognate side of a ssDNA probe, within a given interval. For dsDNA probe it calculate contacts in half the interval on both side (20 kb on one side for ssDNA gives 10 kb on each side for dsDNA)

In [167]:
for sample in samples:
    path = str(str(input_dir_contacts) + str(sample) + str(suffix) + "/")
    sshic_contacts = pd.read_csv(str(path) +  "/classic/" +
                                 str(sample) + str(suffix) +
                                 "_1kb_profile_frequencies.tsv", sep="\t")
    results = np.array([])
    rows = np.array([])
    
    for col in sshic_contacts.columns[3:31]:
        # Get the coordinate and chromosome of the fragment
        fragment_info = reference_table.loc[reference_table['fragment'] == int(col)]
        fragment_coord = fragment_info['coordinate'].values[0]
        fragment_chromosome = fragment_info['chr'].values[0]
        side = fragment_info['side'].values[0]

        #print(fragment_chromosome)
        # Determine the start and end coordinates of the interval based on the side
        if side == 'left':
            interval_start = max(fragment_coord - interval, 0)  # Ensure not to go below 0
            interval_end = fragment_coord
        elif side == 'right':
            # Get the maximum coordinate on the chromosome
            max_chromosome_coord = sshic_contacts.loc[sshic_contacts['chr'] == fragment_chromosome, 'chr_bins'].max()
            interval_start = fragment_coord
            interval_end = min(fragment_coord + interval, max_chromosome_coord)  # Ensure not to exceed chromosome length
        elif side == 'na':
            max_chromosome_coord = sshic_contacts.loc[sshic_contacts['chr'] == fragment_chromosome, 'chr_bins'].max()
            interval_start = max(fragment_coord - (interval/2), 0) 
            interval_end = min(fragment_coord + (interval/2), max_chromosome_coord) 
        else:
            raise ValueError("Side must be 'left' or 'right'")

        # Filter the sshic_contacts table for the specified interval and chromosome
        contacts_within_interval = sshic_contacts[(sshic_contacts['chr'] == fragment_chromosome) &
                                                  (sshic_contacts['chr_bins'] >= interval_start) &
                                                  (sshic_contacts['chr_bins'] <= interval_end)]

        # Sum the contacts for each fragment
        sum_contacts = contacts_within_interval[[col]].sum()
        results = np.append(results, sum_contacts.iloc[0])
        rows = np.append(rows, col)

    results = pd.DataFrame({'Fragment': rows, str('Cis' + str(interval)): results})  
    #results = results.transpose()
    results.to_csv(str(output_dir) + str(sample) + str(suffix) + "_side_specific_cis_" + str(interval) + ".tsv", sep="\t")
    
    
    print(str(sample) + str(suffix) + " is done")   

left
left
left
left
left
left
left
left
left
left
left
left
right
right
right
right
right
right
right
right
right
na
na
na
na
na
na
na
AD162_sub4M_S288c_DSB_LY_Capture_artificial_v8_cutsite_q20_PCRfree is done
left
left
left
left
left
left
left
left
left
left
left
left
right
right
right
right
right
right
right
right
right
na
na
na
na
na
na
na
AD210_sub4M_S288c_DSB_LY_Capture_artificial_v8_cutsite_q20_PCRfree is done
left
left
left
left
left
left
left
left
left
left
left
left
right
right
right
right
right
right
right
right
right
na
na
na
na
na
na
na
AD211_sub4M_S288c_DSB_LY_Capture_artificial_v8_cutsite_q20_PCRfree is done
left
left
left
left
left
left
left
left
left
left
left
left
right
right
right
right
right
right
right
right
right
na
na
na
na
na
na
na
AD233_sub4M_S288c_DSB_LY_Capture_artificial_v8_cutsite_q20_PCRfree is done
left
left
left
left
left
left
left
left
left
left
left
left
right
right
right
right
right
right
right
right
right
na
na
na
na
na
na
na
AD234_sub4M_S288c_DSB_LY_

AD344_sub4M_S288c_DSB_LY_Capture_artificial_v8_cutsite_q20_PCRfree is done
left
left
left
left
left
left
left
left
left
left
left
left
right
right
right
right
right
right
right
right
right
na
na
na
na
na
na
na
AD345_sub4M_S288c_DSB_LY_Capture_artificial_v8_cutsite_q20_PCRfree is done
left
left
left
left
left
left
left
left
left
left
left
left
right
right
right
right
right
right
right
right
right
na
na
na
na
na
na
na
AD346_sub4M_S288c_DSB_LY_Capture_artificial_v8_cutsite_q20_PCRfree is done
left
left
left
left
left
left
left
left
left
left
left
left
right
right
right
right
right
right
right
right
right
na
na
na
na
na
na
na
AD347_sub4M_S288c_DSB_LY_Capture_artificial_v8_cutsite_q20_PCRfree is done
left
left
left
left
left
left
left
left
left
left
left
left
right
right
right
right
right
right
right
right
right
na
na
na
na
na
na
na
AD401_sub4M_S288c_DSB_LY_Capture_artificial_v8_cutsite_q20_PCRfree is done
left
left
left
left
left
left
left
left
left
left
left
left
right
right
right
right
r

left
left
left
left
left
left
right
right
right
right
right
right
right
right
right
na
na
na
na
na
na
na
AD478_sub4M_S288c_DSB_LY_Capture_artificial_v8_cutsite_q20_PCRfree is done
left
left
left
left
left
left
left
left
left
left
left
left
right
right
right
right
right
right
right
right
right
na
na
na
na
na
na
na
AD479_sub4M_S288c_DSB_LY_Capture_artificial_v8_cutsite_q20_PCRfree is done
left
left
left
left
left
left
left
left
left
left
left
left
right
right
right
right
right
right
right
right
right
na
na
na
na
na
na
na
AD480_sub4M_S288c_DSB_LY_Capture_artificial_v8_cutsite_q20_PCRfree is done
left
left
left
left
left
left
left
left
left
left
left
left
right
right
right
right
right
right
right
right
right
na
na
na
na
na
na
na
AD481_sub4M_S288c_DSB_LY_Capture_artificial_v8_cutsite_q20_PCRfree is done
left
left
left
left
left
left
left
left
left
left
left
left
right
right
right
right
right
right
right
right
right
na
na
na
na
na
na
na
AD488_sub4M_S288c_DSB_LY_Capture_artificial_v8_cutsite_

In [150]:
# Define a function to calculate the sum of contacts within a given interval for each fragment
interval = 25000
results = np.array([])
rows = np.array([])

for col in sshic_contacts.columns[3:31]:
    # Get the coordinate and chromosome of the fragment
    fragment_info = reference_table.loc[reference_table['fragment'] == int(col)]
    fragment_coord = fragment_info['coordinate'].values[0]
    fragment_chromosome = fragment_info['chr'].values[0]
    #print(fragment_chromosome)
    # Determine the start and end coordinates of the interval based on the side
    if side == 'left':
        interval_start = max(fragment_coord - interval, 0)  # Ensure not to go below 0
        interval_end = fragment_coord
    elif side == 'right':
        # Get the maximum coordinate on the chromosome
        max_chromosome_coord = sshic_contacts.loc[sshic_contacts['chr'] == fragment_chromosome, 'chr_bins'].max()
        interval_start = fragment_coord
        interval_end = min(fragment_coord + interval, max_chromosome_coord)  # Ensure not to exceed chromosome length
    elif side == 'na':
        max_chromosome_coord = sshic_contacts.loc[sshic_contacts['chr'] == fragment_chromosome, 'chr_bins'].max()
        interval_start = max(fragment_coord - (interval/2), 0) 
        interval_end = min(fragment_coord + (interval/2), max_chromosome_coord) 
    else:
        raise ValueError("Side must be 'left' or 'right'")
    
    # Filter the sshic_contacts table for the specified interval and chromosome
    contacts_within_interval = sshic_contacts[(sshic_contacts['chr'] == fragment_chromosome) &
                                              (sshic_contacts['chr_bins'] >= interval_start) &
                                              (sshic_contacts['chr_bins'] <= interval_end)]
    
    # Sum the contacts for each fragment
    sum_contacts = contacts_within_interval[[col]].sum()
    results = np.append(results, sum_contacts.iloc[0])
    rows = np.append(rows, col)

results = pd.DataFrame({'Fragment': rows, str('Cis' + str(interval)): results})  
#results = results.transpose()

print(results)   

   Fragment  Cis25000
0     74840  0.413515
1     74841  0.410891
2     74842  0.351333
3     74843  0.116667
4     74844  0.173797
5     74845  0.085053
6     74846  0.000000
7     74847  0.070471
8     74848  0.063274
9     74849  0.051383
10    74850  0.033209
11    74851  0.000000
12    74852  0.020833
13    74853  0.035398
14    74854  0.055941
15    74855  0.036323
16    74856  0.036421
17    74857  0.207331
18    74858  0.022223
19    74859  0.083333
20    74860  0.000000
21     5315  0.293364
22     8579  0.260059
23    30726  0.307537
24    32518  0.271096
25    38840  0.260386
26    65856  0.297147
27    68265  0.388481
